# Imports

In [1]:
import warnings

import common_functions
import numpy as np
import pandas as pd
import utils

# Aim of this notebook

**Step 1 - Processing Worker Information from HBM4EU_E-waste_template_V3_all_data_2022_11_23 Excel Workbook:**
1. Process columns: jobs_at_company_site, use_of_PPE, use_of_RPE, enclosed_processing, exhaust_system, and jobs_performed.
2. Determine the number of years worked.
3. Concatenate the above-mentioned columns with the remaining columns to compile data related to the worker group.

**Step 2 - Processing Control Information:**
1. Process control information.

**Step 3 - Merging Worker and Control Dataframes:**
1. Merge the dataframes containing worker and control information.

**Step 4 - Filtering Samples Processed at the Toxcentre, University of Antwerp:**
1. Filter rows to select samples processed at the Toxcentre of the University of Antwerp.

**Step 5 - Preparing Dataframe for POPs Concentrations:**
1. Fill in missing values in the dataframe containing POPs concentrations with LOD/2.

**Step 6 - Concatenating Processed Dataframes:**
1. Concatenate all the processed dataframes to create the final dataset.

**Step 7 - Saving to Disk:**
1. Save thes, please feel free to ask.

# Constructing the main dataframe
## Processing Worker Information
### Process columns: jobs_at_company_site, use_of_PPE, use_of_RPE, enclosed_processing, exhaust_system, and jobs_performed

In [2]:
RAW_DATA_PATH = utils.Configuration.RAW_DATA_PATH.joinpath(
    "HBM4EU_E-waste_template_V3_all_data_2022_11_23.xlsx"
)

In [3]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    jobs_at_company_site = (
        pd.read_excel(
            RAW_DATA_PATH,
            sheet_name="E-waste recycling",
            skiprows=4,
            skipfooter=7,
        )
        .filter(like="Takes place at  the company site")
        .iloc[1:, :]
        .reset_index(drop=True)
        .replace(
            {
                "No": 0,
                "no": 0,
                "Yes": 1,
            }
        )
        .fillna(0)
        .apply(pd.to_numeric)
        .sum(axis=1)
    )

In [4]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    use_of_PPE = (
        pd.read_excel(
            RAW_DATA_PATH,
            sheet_name="E-waste recycling",
            skiprows=4,
            skipfooter=7,
        )
        .filter(like="Use of other PPE")
        .iloc[1:, :]
        .reset_index(drop=True)
        .replace(
            {
                "No": 0,
                "no": 0,
                "Yes": 1,
            }
        )
        .apply(pd.to_numeric)
        .fillna(0)
        .assign(
            sum_other_PPE_use=lambda df_: df_.sum(axis="columns"),
            bool_other_PPE_use=lambda df_: df_.sum_other_PPE_use.mask(
                df_.sum_other_PPE_use > 1, 1
            ),
        )[["sum_other_PPE_use", "bool_other_PPE_use"]]
    )

In [5]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    use_of_RPE = (
        pd.read_excel(
            RAW_DATA_PATH,
            sheet_name="E-waste recycling",
            skiprows=4,
            skipfooter=7,
        )
        .filter(like="Use of RPE")
        .iloc[1:, :]
        .reset_index(drop=True)
        .replace(
            {
                "No": 0,
                "no": 0,
                "Yes": 1,
            }
        )
        .apply(pd.to_numeric)
        .fillna(0)
        .assign(
            sum_RPE_use=lambda df_: df_.sum(axis="columns"),
            bool_RPE_use=lambda df_: df_.sum_RPE_use.mask(df_.sum_RPE_use > 1, 1),
        )[["sum_RPE_use", "bool_RPE_use"]]
    )

In [6]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    enclosed_processing = (
        pd.read_excel(
            RAW_DATA_PATH,
            sheet_name="E-waste recycling",
            skiprows=4,
            skipfooter=7,
        )
        .filter(like="Enclosed process")
        .iloc[1:, :]
        .reset_index(drop=True)
        .replace(
            {
                "No": 0,
                "no": 0,
                "Yes": 1,
            }
        )
        .apply(pd.to_numeric)
        .fillna(0)
        .assign(
            sum_enclosed_process=lambda df_: df_.sum(axis="columns"),
            bool_enclosed_process=lambda df_: df_.sum_enclosed_process.mask(
                df_.sum_enclosed_process > 1, 1
            ),
        )[["sum_enclosed_process", "bool_enclosed_process"]]
    )

In [7]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    exhaust_system = (
        pd.read_excel(
            RAW_DATA_PATH,
            sheet_name="E-waste recycling",
            skiprows=4,
            skipfooter=7,
        )
        .filter(like="Local exhaust ventilation")
        .loc[1:, :"Local exhaust ventilation.14"]
        .reset_index(drop=True)
        .replace(
            {
                "No": 0,
                "no": 0,
                "Yes": 1,
            }
        )
        .apply(pd.to_numeric)
        .fillna(0)
        .assign(
            sum_local_exhaust=lambda df_: df_.sum(axis="columns"),
            bool_local_exhaust=lambda df_: df_.sum_local_exhaust.mask(
                df_.sum_local_exhaust > 1, 1
            ),
        )[["sum_local_exhaust", "bool_local_exhaust"]]
    )

In [8]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    jobs_performed = (
        pd.read_excel(
            RAW_DATA_PATH,
            sheet_name="E-waste recycling",
            skiprows=3,
            skipfooter=7,
        )
        .filter(like="Job")
        .loc[2:, :]
        .reset_index(drop=True)
        .replace(
            {
                "Yes": 1,
                "No": 0,
                "no": 0,
            }
        )
        .rename(columns=lambda x: (x[:6]).replace(":", "").replace(" ", ""))
        .rename(columns={"Operat": "Job1"})
        .fillna(0)
    )

### Determining the number of years worked

In [9]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)

    years_worked_workers = pd.read_excel(
        RAW_DATA_PATH,
        sheet_name="E-waste recycling",
        usecols="JK:KI",
        skiprows=5,
        skipfooter=7,
    )
    years_worked_workers = (
        years_worked_workers.filter(like="Year")
        .pipe(lambda df_: common_functions.col_name_changer(df_, what=".", how="_"))
        .assign(years_worked=lambda df_: common_functions.count_years_worked(df_))
        .loc[:, "years_worked"]
    )

### Read in main workers dataframe and concat with RPE and years_worked

In [10]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    workers_raw = pd.read_excel(
        RAW_DATA_PATH,
        sheet_name="E-waste recycling",
        usecols="B, H, G, GA, GB, GC, GE, GG, GK, GR: GU, GV:HA",
        skiprows=5,
        skipfooter=7,
    )
workers_raw = workers_raw.rename(
    columns={
        "ID": "companyID",
        "ID.1": "ID",
        "Number.30": "height",
        "Number.31": "weight",
        "Hours": "shift_duration",
        "Place.1": "home_location",
        "Yes / No.83": "industrial_plants_in_surroundings",
        "km": "how_many_km",
        "Density": "vehicular_traffic_density",
        "Yes / No.84": "cigarette_smoking",
        "Number.33": "cigarettes_per_day",
        "Duration": "years_smoked",
        "Years.5": "former_smoker_years_ago_stopped",
        "Per day": "former_smoker_cigatette_a_day",
        "Duration.1": "former_smoker_for_how_many_years",
    }
)
workers_raw = pd.concat(
    [
        workers_raw,
        years_worked_workers,
        use_of_PPE,
        use_of_RPE,
        enclosed_processing,
        exhaust_system,
        jobs_performed,
    ],
    axis="columns",
)

workers_raw = workers_raw.assign(
    Category=lambda df_: df_.Category.str.replace(
        r"([0-9])", "", regex=True
    ).str.replace(".", "", regex=True),
    # .astype('category'),
    Age=lambda df_: df_.Age.astype(int),
    height=lambda df_: df_.height.astype(str).str.extract(r"([0-9.]+)").astype(float),
    weight=lambda df_: df_.weight.astype(str).str.extract(r"([0-9.]+)").astype(float),
    BMI=lambda df_: df_.weight.div((df_.height / 100) ** 2),
    how_many_km=lambda df_: df_.how_many_km.replace(r"-", np.nan).astype(float),
).assign(
    how_many_km=lambda df_: df_.how_many_km.mask(df_.how_many_km > 10, np.nan),
    industrial_plants_in_surroundings=lambda df_: df_.industrial_plants_in_surroundings.replace(
        {"Yes": True, "No": False}
    ).mask(
        df_.how_many_km.isna(), df_.industrial_plants_in_surroundings == False
    ),
)

## Processing Control Information

In [11]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    years_worked_controls = pd.read_excel(
        RAW_DATA_PATH,
        sheet_name="Controls",
        usecols="JK:KH",
        skiprows=5,
        # skipfooter = 12
    )
years_worked_controls = (
    years_worked_controls.filter(like="Year")
    .pipe(lambda df_: common_functions.col_name_changer(df_, what=".", how="_"))
    .assign(years_worked=lambda df_: common_functions.count_years_worked(df_))
    .loc[:, "years_worked"]
)

In [12]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)

    controls_raw = pd.read_excel(
        RAW_DATA_PATH,
        sheet_name="Controls",
        usecols="B, G, FZ, GA, GB, GD, GF, GJ,GQ : GZ",
        skiprows=5,
    )
controls_raw = controls_raw.rename(
    columns={
        "ID": "companyID",
        "ID.1": "ID",
        "Number.30": "height",
        "Number.31": "weight",
        "Hours": "shift_duration",
        "Place.1": "home_location",
        "Yes / No.83": "industrial_plants_in_surroundings",
        "km": "how_many_km",
        "Density": "vehicular_traffic_density",
        "Yes / No.84": "cigarette_smoking",
        "Number.33": "cigarettes_per_day",
        "Duration": "years_smoked",
        "Years.5": "former_smoker_years_ago_stopped",
        "Per day": "former_smoker_cigatette_a_day",
        "Duration.1": "former_smoker_for_how_many_years",
    }
)
controls_raw = pd.concat([controls_raw, years_worked_controls], axis="columns")


controls_raw = controls_raw.assign(
    Age=lambda df_: df_.Age.astype(int),
    BMI=lambda df_: df_.weight.div((df_.height / 100) ** 2),
    how_many_km=lambda df_: df_.how_many_km.astype(str)
    .str.extract(r"([0-9.]+)")
    .astype(float),
).assign(
    how_many_km=lambda df_: df_.how_many_km.mask(df_.how_many_km > 10, np.nan),
    industrial_plants_in_surroundings=lambda df_: df_.industrial_plants_in_surroundings.replace(
        {"Yes": True, "No": False}
    ).mask(
        df_.how_many_km.isna(), df_.industrial_plants_in_surroundings == False
    ),
)

##  Merging Worker and Control Dataframes

In [13]:
workers_controls_all = pd.concat([workers_raw, controls_raw], ignore_index=True)
print(workers_controls_all.shape)

(268, 44)


C:\Users\s0212777\AppData\Local\Temp\ipykernel_14980\1503741716.py:1: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  workers_controls_all = pd.concat([workers_raw, controls_raw], ignore_index=True)


### Filtering Samples Processed at the Toxcentre, University of Antwerp

In [14]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    UA_measured_samples = pd.read_excel(
        RAW_DATA_PATH,
        sheet_name="E-waste study UA data",
        usecols="B:D",
        skiprows=6,
        skipfooter=24,
    )

UA_measured_samples = UA_measured_samples.rename(
    columns={
        "ID short": "ID",
        "Unnamed: 2": "main_category",
        "Unnamed: 3": "CTR_subcategory",
    }
).assign(
    CTR_subcategory=lambda df_: df_.CTR_subcategory.replace(
        {1: "within_CTR", 2: "outwith_CTR"}
    )
)

In [15]:
# only keep rows (IDs) that are present in the UA_measured_samples, ie measured by us

workers_controls_all_measured = workers_controls_all[
    workers_controls_all["ID"].isin(UA_measured_samples["ID"])
]
print(workers_controls_all_measured.shape)

(169, 44)


In [16]:
workers_controls_all_measured = (
    workers_controls_all_measured.set_index("ID")
    .join(UA_measured_samples.set_index("ID"))
    .assign(
        sub_category=lambda df_: df_.Category.mask(
            df_.Category.isna(), df_.CTR_subcategory
        )
    )
    .drop(["Category", "CTR_subcategory"], axis="columns")
    .reset_index(drop=False)
)
print(workers_controls_all_measured.shape)

(169, 45)


## Preparing Dataframe for POPs Concentrations

In [17]:
UA_POP_raw = pd.read_excel(
    RAW_DATA_PATH,
    sheet_name="E-waste study UA data",
    skiprows=3,
    nrows=172,
)

columns_to_keep = [
    "Worker ID (without letter 'E')",
    "PCB 28",
    "PCB 52",
    "PCB 101",
    "PCB 118",
    "PCB 138",
    "PCB 153",
    "PCB 180",
    "BDE 28 [2,4,4′-Tribromodiphenyl ether]",
    "BDE 47 [2,2′,4,4′-tetrabromodiphenyl ether]",
    "BDE 99",
    "BDE 100",
    "BDE 153",
    "BDE 154",
    "BDE 183",
    "BDE 209",
    "Dechlorane",
]

UA_POP_raw = (
    UA_POP_raw.loc[3:, lambda df_: ~df_.columns.str.contains("Unnamed")]
    .reset_index(drop=True)
    .loc[:, lambda df_: df_.columns.isin(columns_to_keep)]
    .rename(
        columns={
            "BDE 28 [2,4,4′-Tribromodiphenyl ether]": "BDE 28",
            "BDE 47 [2,2′,4,4′-tetrabromodiphenyl ether]": "BDE 47",
        }
    )
)

### Fill in missing values in the dataframe containing POPs concentrations with LOD/2

In [18]:
keys = UA_POP_raw.columns[1:]
value = []
replaced_dict = {key: list(value) for key in keys}

UA_POP_mod = UA_POP_raw.copy(deep=True)
UA_POP_mod

for i in UA_POP_mod.columns[1:]:
    for x in UA_POP_mod[i]:
        if isinstance(x, str):
            replaced_dict[i].append(float(x.replace("<", "")) / 2)
        else:
            replaced_dict[i].append(x)


UA_POP_mod = pd.concat(
    [
        UA_POP_mod["Worker ID (without letter 'E')"],
        (pd.DataFrame.from_dict(replaced_dict, orient="columns")),
    ],
    axis="columns",
)

## Concatenating Processed Dataframes

In [19]:
df = (
    workers_controls_all_measured.set_index("ID")
    .join(UA_POP_mod.set_index("Worker ID (without letter 'E')"))
    .assign(
        main_category=lambda df_: df_.main_category.replace(
            {"Worker (E-waste recycling)": "Worker"}
        ),
        sub_category=lambda df_: df_.sub_category.replace(" ", "", regex=True).replace(
            {
                "Browngoods": "Brown goods",
                "Metalsandplastics": "Metals and plastics",
                "MiscellaneousE-waste": "Miscellaneous",
                "Whitegoods": "White goods",
            }
        ),
        Tonnes=lambda df: pd.to_numeric(df.Tonnes, errors="coerce"),
    )
)

df.loc[:, "PCB 28":"Dechlorane"] = df.loc[:, "PCB 28":"Dechlorane"].mul(
    1000
)  # multiplied by 1000 to convert the concentrations to ng/L
print(df.shape)

(169, 60)


# Saving to Disk

In [20]:
df.to_parquet(
    utils.Configuration.INTERIM_DATA_PATH.joinpath(
        "HBM4EU_E-waste_template_V3_all_data_INTERIM.parquet.gzip"
    ),
    compression="gzip",
)